In [31]:
!pip3 install transformers
!pip3 install pytorch-lightning
!pip3 install sentence_transformers

In [32]:
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers import losses, datasets
import sentence_transformers
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from math import floor
from sklearn.model_selection import train_test_split
import random
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [34]:
sampls = []
for line in open('/content/phrases.txt', errors="ignore"):
    f1 = line.rstrip('\n')
    sampls.append(f1)
print(len(sampls))

192


In [53]:
def add_noise(phrase):
    phrase_arr = phrase.split(" ")
    indexs_arr = list(range(len(phrase_arr)))
    try:
        indexs, _ = train_test_split(indexs_arr, train_size=0.4, shuffle=True)
    except ValueError:
        return phrase
    indexs.sort()
    phrase_f = []
    for index in indexs:
        phrase_f.append(phrase_arr[index])
    return ' '.join(phrase_f)

In [49]:
print(add_noise("Contamination through cracks in the reservoir most  probably occurred during heavy rainfall"))
print(add_noise("Contamination through cracks in the reservoir most  probably occurred during heavy rainfall"))
print(add_noise("Contamination through cracks in the reservoir most  probably occurred during heavy rainfall"))

Contamination reservoir most occurred heavy
Contamination through reservoir during heavy
through cracks the probably during


In [54]:
train_dataset1 = datasets.DenoisingAutoEncoderDataset(sampls, add_noise)
train_dataset2 = datasets.DenoisingAutoEncoderDataset(sampls, add_noise)
train_dataloader1 = DataLoader(train_dataset1, batch_size=4, shuffle=True)
train_dataloader2 = DataLoader(train_dataset2, batch_size=4, shuffle=True)

train_loss = losses.DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)

model.fit(
    train_objectives=[(train_dataloader1, train_loss)],
    epochs=5,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

model.fit(
    train_objectives=[(train_dataloader2, train_loss)],
    epochs=5,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)


Some weights of BertLMHeadModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/ and are newly initialized: ['encoder.layer.3.crossattention.self.query.bias', 'encoder.layer.0.crossattention.self.key.bias', 'encoder.layer.5.crossattention.self.query.bias', 'encoder.layer.5.crossattention.self.key.bias', 'cls.predictions.bias', 'encoder.layer.5.crossattention.output.LayerNorm.bias', 'encoder.layer.2.crossattention.self.query.bias', 'encoder.layer.3.crossattention.self.value.weight', 'encoder.layer.4.crossattention.output.dense.bias', 'encoder.layer.2.crossattention.self.key.bias', 'encoder.layer.3.crossattention.output.LayerNorm.weight', 'encoder.layer.4.crossattention.output.LayerNorm.bias', 'encoder.layer.2.crossattention.output.dense.weight', 'encoder.layer.1.crossattention.self.query.bias', 'encoder.layer.4.crossattention.self.value.bias', 'encoder.layer.4.crossattention.self.query.weight', 'encoder.la

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/48 [00:00<?, ?it/s]

Iteration:   0%|          | 0/48 [00:00<?, ?it/s]

Iteration:   0%|          | 0/48 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/48 [00:00<?, ?it/s]

Iteration:   0%|          | 0/48 [00:00<?, ?it/s]

Iteration:   0%|          | 0/48 [00:00<?, ?it/s]

Iteration:   0%|          | 0/48 [00:00<?, ?it/s]

Iteration:   0%|          | 0/48 [00:00<?, ?it/s]

In [55]:
model.save('/content/drive/MyDrive/Uni/model')